In [1]:
from fastai.vision import *
from fastai.metrics import error_rate

In [2]:
import json

In [3]:
import pandas as pd

In [4]:
p = Path('../data')

In [5]:
j = json.load(open('../train.json'))

In [6]:
df = pd.DataFrame({'fnames': list(p.iterdir())})

In [7]:
df.shape

(1014069, 1)

In [8]:
df['image_id'] = df.fnames.apply(lambda x: int(str(x).split('/')[-1].split('.')[0]))

In [9]:
anno = {int(k['imageId']): k['labelId'] for k in j['annotations']}

In [10]:
df['cats'] = df.image_id.apply(lambda x: anno[x])

In [11]:
# df['cats'] = df['cats'].apply(lambda x: [int(i) for i in x])

In [12]:
df['cats'] = df['cats'].apply(lambda x: ', '.join(x))

In [13]:
df.head()

,fnames,image_id,cats
0,../data/493058.jpg,493058,"183, 131, 66, 128"
1,../data/621835.jpg,621835,"100, 222, 66"
2,../data/77531.jpg,77531,"15, 66, 137, 214"
3,../data/337719.jpg,337719,"26, 106, 49, 28"
4,../data/491437.jpg,491437,"18, 66, 116, 105"


In [14]:
import jpeg4py as jpeg

In [15]:
def imread_fast(fname):
    """
    same as imread or Image.open but 6x faster
    installations: > sudo apt-get install libturbojpeg
                   > pip install jpeg4py
    input : file path as string
    output: numpy array (HxWx3)
    """
    return jpeg.JPEG(fname).decode().shape

In [16]:
# df['shapes'] = df['fnames'].apply(lambda x: imread_fast('../data/' + x))

In [17]:
df['fnames'] = df.fnames.astype('str')

In [18]:
df['fnames'] = df.fnames.apply(lambda x: x.split('/')[-1])

In [19]:
df[['fnames', 'cats']].to_csv('../MC.csv', index=False)

In [20]:
!head ../MC.csv

fnames,cats
493058.jpg,"183, 131, 66, 128"
621835.jpg,"100, 222, 66"
77531.jpg,"15, 66, 137, 214"
337719.jpg,"26, 106, 49, 28"
491437.jpg,"18, 66, 116, 105"
1005540.jpg,"66, 98, 214, 171, 175"
511680.jpg,"220, 66, 203, 186, 180, 150"
565052.jpg,"62, 66, 102, 53, 214, 105, 218"
626342.jpg,"93, 106, 28"


In [21]:
data = ImageDataBunch.from_csv('../', 'data', label_delim=',', 
                               csv_labels='MC.csv' ,label_col='cats', size=224)

In [21]:
# data = ImageDataBunch.from_df(df=df, path='../data', label_delim=',', label_col='cats', no_check=True)

In [22]:
data

ImageDataBunch;

Train: LabelList (811256 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: MultiCategoryList
183; 131; 66; 128,15; 66; 137; 214,26; 106; 49; 28,18; 66; 116; 105,66; 98; 214; 171; 175
Path: ..;

Valid: LabelList (202813 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: MultiCategoryList
21; 131; 17; 66; 98; 214; 105,189; 66,18; 17; 66; 44; 98; 153; 171; 128,95; 113; 137,51; 17; 66
Path: ..;

Test: None

In [23]:
x, y = next(iter(data.valid_dl))

In [24]:
x.cuda()

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [43]:
x.shape

torch.Size([64, 3, 224, 224])

In [44]:
y.shape

torch.Size([64, 252])

In [45]:
y

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [46]:
f_model=models.resnet34
sz=224
bs=64

In [51]:
learn = cnn_learner(data, f_model)

In [52]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ubuntu/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._

KeyboardInterrupt: 